# Diploma project - Prediction of Gas Leaks in main Gas Pipelines of RK

In [1]:
#import requiered libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st
from sklearn.model_selection import train_test_split, GridSearchCV, KFold,cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from pyngrok import ngrok

C:\Program Files (x86)\sabina\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
#pip install pyngrok

In [3]:
#pip install xgboost

In [4]:
#pip install xgboost==1.5.0

In [5]:
nitka = pd.read_csv('_Line_MG_Nitka.csv')
nitka.head()
# dataset of pipeline objects with their characteristics 

,Id,DateCreate,DateUpdate,IsDeleted,Nazvanie,Sokrash_nazvanie,Kilometr_nachala_nitki,Kilometr_okoncha_nitki,Dlina,Davlenie_proektnoe,...,FileId,Primechanie,UserId,ParentEntityId,ParentId,GisId,IsTuran,IsChecked,RefServiceId,SapId
0,14,2019-08-27 05:20:43,2019-08-28 09:22:45,False,"МГ ""САЦ-5""и ""САЦ-3""",NaN,311.0,1134.0,823.0,NaN,...,a57fc7d3-0ed6-4f43-ae73-018696c136dd,Опорненское ЛПУ,2,15936552,10,NaN,False,False,NaN,NaN
1,12,2019-08-27 05:20:43,2019-11-19 06:26:53,False,2 нитка. МГ САЦ-4,NaN,41.0,104.4,63.4,NaN,...,02e7df09-197b-48e9-a02e-b8250a8bfd1b,NaN,2,15936552,8,NaN,False,False,NaN,NaN
2,11,2019-08-27 05:20:43,2019-11-19 08:58:42,False,1 нитка. МГ САЦ-4,NaN,41.0,1134.0,1093.0,NaN,...,f4671292-6594-4dd2-833b-757fcf002487,NaN,2,15936552,8,NaN,False,False,NaN,NaN
3,23,2019-08-27 05:20:44,2019-08-28 09:04:10,False,"Перемычка между. МГ""САЦ -2"" и ""САЦ-3"". ...",NaN,502.0,NaN,NaN,NaN,...,1c26d9d4-72b0-486d-8b50-6a98164c3a5c,Опорненское ЛПУ,2,15936552,15,NaN,False,False,NaN,NaN
4,25,2019-08-27 05:20:44,2019-08-28 09:40:15,False,САЦ-4,NaN,880.0,1134.0,254.0,NaN,...,4b969e51-8824-465a-b2d5-c56388aed0e1,Джангалинское ЛПУ,2,15936552,8,NaN,False,False,NaN,NaN


In [6]:
leak_info = pd.read_csv('Gas_leak_params.csv', sep='\t', encoding='utf-8')
leak_info.head()
# dataset with leak data records with their parameters

,pipeline_name,Km,press_start,press_end,temp_start,temp_end,diameter,gas_consum_start,gas_consum_end,leak_dist_X0
0,"МГ ""Макат-Северный Кавказ""",371.8,40.0,45.0,15.0,25.0,1420,5.0,15.0,277.0
1,"МГ ""Макат-Северный Кавказ""",371.8,39.1,15.0,12.0,16.0,1420,5.7,4.2,0.0
2,"МГ ""Макат-Северный Кавказ""",371.8,21.2,20.1,10.8,10.1,1420,3.1,4.7,0.0
3,"МГ ""Макат-Северный Кавказ""",371.8,9.8,8.6,6.0,7.1,1420,4.9,6.4,215.0
4,"МГ ""Макат-Северный Кавказ""",371.8,50.5,45.9,16.1,11.0,1420,22.9,20.0,0.0


In [7]:
merged_nitka_leak_info = pd.merge(left=nitka, right=leak_info, left_on='Nazvanie', right_on='pipeline_name')
merged_nitka_leak_info
# merging two datasets to get one full informative set of pipelines

,Id,DateCreate,DateUpdate,IsDeleted,Nazvanie,Sokrash_nazvanie,Kilometr_nachala_nitki,Kilometr_okoncha_nitki,Dlina,Davlenie_proektnoe,...,pipeline_name,Km,press_start,press_end,temp_start,temp_end,diameter,gas_consum_start,gas_consum_end,leak_dist_X0
0,25,2019-08-27 05:20:44,2019-08-28 09:40:15,False,САЦ-4,NaN,880.0,1134.0,254.0,NaN,...,САЦ-4,823.0,66.3,78.9,21.2,10.2,1420,2.5,17.4,503.0
1,25,2019-08-27 05:20:44,2019-08-28 09:40:15,False,САЦ-4,NaN,880.0,1134.0,254.0,NaN,...,САЦ-4,823.0,66.5,74.6,55.2,18.9,1420,23.0,6.5,35.0
2,25,2019-08-27 05:20:44,2019-08-28 09:40:15,False,САЦ-4,NaN,880.0,1134.0,254.0,NaN,...,САЦ-4,823.0,75.4,74.7,87.8,67.5,1420,11.3,40.2,0.0
3,25,2019-08-27 05:20:44,2019-08-28 09:40:15,False,САЦ-4,NaN,880.0,1134.0,254.0,NaN,...,САЦ-4,823.0,3.2,16.0,59.8,29.3,1420,18.3,14.2,0.0
4,25,2019-08-27 05:20:44,2019-08-28 09:40:15,False,САЦ-4,NaN,880.0,1134.0,254.0,NaN,...,САЦ-4,823.0,64.2,20.3,53.9,86.8,1420,21.5,12.9,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,4,2019-08-27 05:20:42,2021-06-21 13:49:15,False,Основная нитка. МГ Жанажол - КС13,NaN,NaN,NaN,157.4,NaN,...,Основная нитка. МГ Жанажол - КС13,157.4,75.2,53.2,98.0,20.9,813,24.0,46.0,0.0
996,4,2019-08-27 05:20:42,2021-06-21 13:49:15,False,Основная нитка. МГ Жанажол - КС13,NaN,NaN,NaN,157.4,NaN,...,Основная нитка. МГ Жанажол - КС13,157.4,54.2,15.1,94.0,86.8,813,1.6,3.0,0.0
997,4,2019-08-27 05:20:42,2021-06-21 13:49:15,False,Основная нитка. МГ Жанажол - КС13,NaN,NaN,NaN,157.4,NaN,...,Основная нитка. МГ Жанажол - КС13,157.4,33.1,21.1,46.0,11.6,813,42.6,42.2,0.0
998,4,2019-08-27 05:20:42,2021-06-21 13:49:15,False,Основная нитка. МГ Жанажол - КС13,NaN,NaN,NaN,157.4,NaN,...,Основная нитка. МГ Жанажол - КС13,157.4,7.9,49.9,63.1,43.3,813,25.9,26.9,0.0


## Data preprocessing

In [8]:
data = merged_nitka_leak_info.drop(['Nazvanie','Dlina'], axis = 1) #remove duplicated columns that we used to merge 
data

,Id,DateCreate,DateUpdate,IsDeleted,Sokrash_nazvanie,Kilometr_nachala_nitki,Kilometr_okoncha_nitki,Davlenie_proektnoe,Diametr_nomenalny,RefDiametr_po_tpId,...,pipeline_name,Km,press_start,press_end,temp_start,temp_end,diameter,gas_consum_start,gas_consum_end,leak_dist_X0
0,25,2019-08-27 05:20:44,2019-08-28 09:40:15,False,NaN,880.0,1134.0,NaN,NaN,4159.0,...,САЦ-4,823.0,66.3,78.9,21.2,10.2,1420,2.5,17.4,503.0
1,25,2019-08-27 05:20:44,2019-08-28 09:40:15,False,NaN,880.0,1134.0,NaN,NaN,4159.0,...,САЦ-4,823.0,66.5,74.6,55.2,18.9,1420,23.0,6.5,35.0
2,25,2019-08-27 05:20:44,2019-08-28 09:40:15,False,NaN,880.0,1134.0,NaN,NaN,4159.0,...,САЦ-4,823.0,75.4,74.7,87.8,67.5,1420,11.3,40.2,0.0
3,25,2019-08-27 05:20:44,2019-08-28 09:40:15,False,NaN,880.0,1134.0,NaN,NaN,4159.0,...,САЦ-4,823.0,3.2,16.0,59.8,29.3,1420,18.3,14.2,0.0
4,25,2019-08-27 05:20:44,2019-08-28 09:40:15,False,NaN,880.0,1134.0,NaN,NaN,4159.0,...,САЦ-4,823.0,64.2,20.3,53.9,86.8,1420,21.5,12.9,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,4,2019-08-27 05:20:42,2021-06-21 13:49:15,False,NaN,NaN,NaN,NaN,NaN,4162.0,...,Основная нитка. МГ Жанажол - КС13,157.4,75.2,53.2,98.0,20.9,813,24.0,46.0,0.0
996,4,2019-08-27 05:20:42,2021-06-21 13:49:15,False,NaN,NaN,NaN,NaN,NaN,4162.0,...,Основная нитка. МГ Жанажол - КС13,157.4,54.2,15.1,94.0,86.8,813,1.6,3.0,0.0
997,4,2019-08-27 05:20:42,2021-06-21 13:49:15,False,NaN,NaN,NaN,NaN,NaN,4162.0,...,Основная нитка. МГ Жанажол - КС13,157.4,33.1,21.1,46.0,11.6,813,42.6,42.2,0.0
998,4,2019-08-27 05:20:42,2021-06-21 13:49:15,False,NaN,NaN,NaN,NaN,NaN,4162.0,...,Основная нитка. МГ Жанажол - КС13,157.4,7.9,49.9,63.1,43.3,813,25.9,26.9,0.0


In [9]:
data.columns

Index(['Id', 'DateCreate', 'DateUpdate', 'IsDeleted', 'Sokrash_nazvanie',
       'Kilometr_nachala_nitki', 'Kilometr_okoncha_nitki',
       'Davlenie_proektnoe', 'Diametr_nomenalny', 'RefDiametr_po_tpId',
       'RefTip_prokladki_trubyId', 'Data_vvoda_v_ekspluata',
       'RefTehnicheskoe_sostoyanieId', 'FileId', 'Primechanie', 'UserId',
       'ParentEntityId', 'ParentId', 'GisId', 'IsTuran', 'IsChecked',
       'RefServiceId', 'SapId', 'pipeline_name', 'Km', 'press_start',
       'press_end', 'temp_start', 'temp_end', 'diameter', 'gas_consum_start',
       'gas_consum_end', 'leak_dist_X0'],
      dtype='object')

In [10]:
data=data.drop(['Id','DateCreate','DateUpdate','IsDeleted','Sokrash_nazvanie','Kilometr_nachala_nitki','Kilometr_okoncha_nitki',
           'FileId', 'Primechanie', 'UserId', 'ParentEntityId','GisId','IsTuran','IsChecked', 'RefServiceId','SapId','ParentId',
                  'RefDiametr_po_tpId','RefTip_prokladki_trubyId','Data_vvoda_v_ekspluata','RefTehnicheskoe_sostoyanieId'],
               axis=1)
data
#drop unnecessary columns bc most of them using in system for connection with other tables in database

,Davlenie_proektnoe,Diametr_nomenalny,pipeline_name,Km,press_start,press_end,temp_start,temp_end,diameter,gas_consum_start,gas_consum_end,leak_dist_X0
0,NaN,NaN,САЦ-4,823.0,66.3,78.9,21.2,10.2,1420,2.5,17.4,503.0
1,NaN,NaN,САЦ-4,823.0,66.5,74.6,55.2,18.9,1420,23.0,6.5,35.0
2,NaN,NaN,САЦ-4,823.0,75.4,74.7,87.8,67.5,1420,11.3,40.2,0.0
3,NaN,NaN,САЦ-4,823.0,3.2,16.0,59.8,29.3,1420,18.3,14.2,0.0
4,NaN,NaN,САЦ-4,823.0,64.2,20.3,53.9,86.8,1420,21.5,12.9,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,NaN,Основная нитка. МГ Жанажол - КС13,157.4,75.2,53.2,98.0,20.9,813,24.0,46.0,0.0
996,NaN,NaN,Основная нитка. МГ Жанажол - КС13,157.4,54.2,15.1,94.0,86.8,813,1.6,3.0,0.0
997,NaN,NaN,Основная нитка. МГ Жанажол - КС13,157.4,33.1,21.1,46.0,11.6,813,42.6,42.2,0.0
998,NaN,NaN,Основная нитка. МГ Жанажол - КС13,157.4,7.9,49.9,63.1,43.3,813,25.9,26.9,0.0


In [11]:
data.info()  #to know features data types

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Davlenie_proektnoe  689 non-null    float64
 1   Diametr_nomenalny   0 non-null      float64
 2   pipeline_name       1000 non-null   object 
 3   Km                  1000 non-null   float64
 4   press_start         1000 non-null   float64
 5   press_end           1000 non-null   float64
 6   temp_start          1000 non-null   float64
 7   temp_end            1000 non-null   float64
 8   diameter            1000 non-null   int64  
 9   gas_consum_start    1000 non-null   float64
 10  gas_consum_end      1000 non-null   float64
 11  leak_dist_X0        1000 non-null   float64
dtypes: float64(10), int64(1), object(1)
memory usage: 101.6+ KB


In [12]:
#data['pipeline_name']=data['pipeline_name'].factorize() 
data['pipeline_name'] = pd.factorize(data['pipeline_name'])[0]
data

,Davlenie_proektnoe,Diametr_nomenalny,pipeline_name,Km,press_start,press_end,temp_start,temp_end,diameter,gas_consum_start,gas_consum_end,leak_dist_X0
0,NaN,NaN,0,823.0,66.3,78.9,21.2,10.2,1420,2.5,17.4,503.0
1,NaN,NaN,0,823.0,66.5,74.6,55.2,18.9,1420,23.0,6.5,35.0
2,NaN,NaN,0,823.0,75.4,74.7,87.8,67.5,1420,11.3,40.2,0.0
3,NaN,NaN,0,823.0,3.2,16.0,59.8,29.3,1420,18.3,14.2,0.0
4,NaN,NaN,0,823.0,64.2,20.3,53.9,86.8,1420,21.5,12.9,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,NaN,10,157.4,75.2,53.2,98.0,20.9,813,24.0,46.0,0.0
996,NaN,NaN,10,157.4,54.2,15.1,94.0,86.8,813,1.6,3.0,0.0
997,NaN,NaN,10,157.4,33.1,21.1,46.0,11.6,813,42.6,42.2,0.0
998,NaN,NaN,10,157.4,7.9,49.9,63.1,43.3,813,25.9,26.9,0.0


In [13]:
data

,Davlenie_proektnoe,Diametr_nomenalny,pipeline_name,Km,press_start,press_end,temp_start,temp_end,diameter,gas_consum_start,gas_consum_end,leak_dist_X0
0,NaN,NaN,0,823.0,66.3,78.9,21.2,10.2,1420,2.5,17.4,503.0
1,NaN,NaN,0,823.0,66.5,74.6,55.2,18.9,1420,23.0,6.5,35.0
2,NaN,NaN,0,823.0,75.4,74.7,87.8,67.5,1420,11.3,40.2,0.0
3,NaN,NaN,0,823.0,3.2,16.0,59.8,29.3,1420,18.3,14.2,0.0
4,NaN,NaN,0,823.0,64.2,20.3,53.9,86.8,1420,21.5,12.9,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,NaN,10,157.4,75.2,53.2,98.0,20.9,813,24.0,46.0,0.0
996,NaN,NaN,10,157.4,54.2,15.1,94.0,86.8,813,1.6,3.0,0.0
997,NaN,NaN,10,157.4,33.1,21.1,46.0,11.6,813,42.6,42.2,0.0
998,NaN,NaN,10,157.4,7.9,49.9,63.1,43.3,813,25.9,26.9,0.0


In [14]:
data.isna().sum() #checking for missing values

Davlenie_proektnoe     311
Diametr_nomenalny     1000
pipeline_name            0
Km                       0
press_start              0
press_end                0
temp_start               0
temp_end                 0
diameter                 0
gas_consum_start         0
gas_consum_end           0
leak_dist_X0             0
dtype: int64

In [15]:
data = data.drop(['Diametr_nomenalny'], axis = 1) #remove unusefull column bc it doesnt have any values
data['Davlenie_proektnoe'] = data['Davlenie_proektnoe'].fillna(data['Davlenie_proektnoe'].mean()) #fill nan with mean value

In [16]:
data.isna().sum() #checking for missing values

Davlenie_proektnoe    0
pipeline_name         0
Km                    0
press_start           0
press_end             0
temp_start            0
temp_end              0
diameter              0
gas_consum_start      0
gas_consum_end        0
leak_dist_X0          0
dtype: int64

In [17]:
data.to_csv('data.csv', index = False)

In [18]:
data

,Davlenie_proektnoe,pipeline_name,Km,press_start,press_end,temp_start,temp_end,diameter,gas_consum_start,gas_consum_end,leak_dist_X0
0,5.779826,0,823.0,66.3,78.9,21.2,10.2,1420,2.5,17.4,503.0
1,5.779826,0,823.0,66.5,74.6,55.2,18.9,1420,23.0,6.5,35.0
2,5.779826,0,823.0,75.4,74.7,87.8,67.5,1420,11.3,40.2,0.0
3,5.779826,0,823.0,3.2,16.0,59.8,29.3,1420,18.3,14.2,0.0
4,5.779826,0,823.0,64.2,20.3,53.9,86.8,1420,21.5,12.9,0.0
...,...,...,...,...,...,...,...,...,...,...,...
995,5.779826,10,157.4,75.2,53.2,98.0,20.9,813,24.0,46.0,0.0
996,5.779826,10,157.4,54.2,15.1,94.0,86.8,813,1.6,3.0,0.0
997,5.779826,10,157.4,33.1,21.1,46.0,11.6,813,42.6,42.2,0.0
998,5.779826,10,157.4,7.9,49.9,63.1,43.3,813,25.9,26.9,0.0


In [ ]:
data.describe() #getting statistics of numerical columns 

In [ ]:
fig, ax = plt.subplots()
sns.histplot(data['leak_dist_X0'],color='c',ax=ax,kde=True, stat="density", linewidth=0)
ax.set(title='Target feature distribution') #plot distribution of target variable

In [ ]:
# visualize nominal_features with distribution plots
f, ax = plt.subplots(9, figsize=(10,20))
sns.histplot(data['Davlenie_proektnoe'],color='c',ax=ax[0],kde=True, stat="density", linewidth=0)
sns.histplot(data['Km'],color='c',ax=ax[1],kde=True, stat="density", linewidth=0)
sns.histplot(data['press_start'],color='c',ax=ax[2],kde=True, stat="density", linewidth=0)
sns.histplot(data['press_end'],color='c',ax=ax[3],kde=True, stat="density", linewidth=0)
sns.histplot(data['temp_start'],color='c',ax=ax[4],kde=True, stat="density", linewidth=0)
sns.histplot(data['temp_end'],color='c',ax=ax[5],kde=True, stat="density", linewidth=0)
sns.histplot(data['diameter'],color='c',ax=ax[6],kde=True, stat="density", linewidth=0)
sns.histplot(data['gas_consum_start'],color='c',ax=ax[7],kde=True, stat="density", linewidth=0)
sns.histplot(data['gas_consum_end'],color='c',ax=ax[8],kde=True, stat="density", linewidth=0)
ax[0].set_title('projected pressure')
ax[1].set_title('length')
ax[2].set_title('press start')
ax[3].set_title('press end')
ax[4].set_title('temperature start')
ax[5].set_title('temperatue end')
ax[6].set_title('diameter')
ax[7].set_title('gas consumption start')
ax[8].set_title('gas consumption end')

In [ ]:
# visualize pressure start rate with distance to leak
plt.scatter(data["press_start"],data["leak_dist_X0"],color='r',marker='+')
plt.title('pressure start rate and Distance to leak')
plt.xlabel('pressure start rate')
plt.ylabel('Distance to leak')
plt.grid(False)

In [ ]:
# visualize pressure end rate with distance to leak
plt.scatter(data["press_end"],data["leak_dist_X0"],color='r',marker='+')
plt.title('pressure ending rate and Distance to leak')
plt.xlabel('pressure end rate')
plt.ylabel('Distance to leak')
plt.grid(False)

In [ ]:
# visualize temperature start with distance to leak
plt.scatter(data["temp_start"],data["leak_dist_X0"],color='r',marker='+')
plt.title('Temperature start rate and Distance to leak')
plt.xlabel('Temperature start')
plt.ylabel('Distance to leak')
plt.grid(False)

In [ ]:
# visualize temperature end with distance to leak
plt.scatter(data["temp_end"],data["leak_dist_X0"],color='r',marker='+')
plt.title('temperature ending rate and Distance to leak')
plt.xlabel('temperature end')
plt.ylabel('Distance to leak')
plt.grid(False)

In [ ]:
# visualize gas_consum_start with distance to leak
plt.scatter(data["gas_consum_start"],data["leak_dist_X0"],color='r',marker='+')
plt.title('Gas consumption start rate and Distance to leak')
plt.xlabel('Gas consumption start')
plt.ylabel('Distance to leak')
plt.grid(False)

In [ ]:
# visualize gas_consum_end with distance to leak
plt.scatter(data["gas_consum_end"],data["leak_dist_X0"],color='r',marker='+')
plt.title('Gas consumption ending rate and Distance to leak')
plt.xlabel('Gas consumption end')
plt.ylabel('Distance to leak')
plt.grid(False)

In [ ]:
# visualize gas_consum_end with 
plt.scatter(data["Km"],data["leak_dist_X0"],color='r',marker='+')
plt.title('Kilometer and Distance to leak')
plt.xlabel('Kilometer')
plt.ylabel('Distance to leak')
plt.grid(False)

In [ ]:
# visualize gas_consum_end with 
plt.scatter(data["diameter"],data["leak_dist_X0"],color='r',marker='+')
plt.title('Diameter and Distance to leak')
plt.xlabel('Diameter')
plt.ylabel('Distance to leak')
plt.grid(False)

In [ ]:
# visualize projected pressure with distance to leak
plt.scatter(data["Davlenie_proektnoe"],data["leak_dist_X0"],color='r',marker='+')
plt.title('projected pressure and Distance to leak')
plt.xlabel('projected pressure')
plt.ylabel('Distance to leak')
plt.grid(False)

## Model building

In [ ]:
data = data.drop(['pipeline_name'],axis=1) #remove object feature
data

In [ ]:
X = data.drop('leak_dist_X0',axis=1)
y = data.leak_dist_X0

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.3) #splitting to train and test sets

In [ ]:
#firstly, we will try simple linear regression model for experiment
clf = LinearRegression()
clf.fit(X_train,y_train)
print('Training score: {:0.3f}'.format(r2_score(y_train,clf.predict(X_train))))
print('Training MSE: {:0.3f}'.format(mean_squared_error(y_train,clf.predict(X_train))))
print('test set score: {:0.3f}'.format(r2_score(y_test,clf.predict(X_test))))
print('test set MSE: {:0.3f}'.format(mean_squared_error(y_test,clf.predict(X_test))))
print('Coefficients: {}\n Intercept: {}'.format(clf.coef_,clf.intercept_))

as we see in results, because of non linear correlation between some independent variables and target, linear regression model cannot be applied to our data, instead of this we are going to use random forest regression algortihm 

"Decision Trees and its ensemble - Random Forest are great for obtaining non-linear relationships between input features and the target variable" - [https://neptune.ai/blog/random-forest-regression-when-does-it-fail-and-why] 

its also no matter if features are scaled or not for this algorithm

In [ ]:
rfr = RandomForestRegressor(max_depth=2, random_state=42)
rfr.fit(X_train,y_train)
print('Training score: {:0.3f}'.format(r2_score(y_train,rfr.predict(X_train))))
print('Training MSE: {:0.3f}'.format(mean_squared_error(y_train,rfr.predict(X_train))))
print('test set score: {:0.3f}'.format(r2_score(y_test,rfr.predict(X_test))))
print('test set MSE: {:0.3f}'.format(mean_squared_error(y_test,rfr.predict(X_test))))

In results after implementing Random forest for our data, accuracies of traing and test sets are lower than we could expect so that to increase them, one of the methods called Hyperparameter tuning of model can be deployed. "Therefore, the standard procedure for hyperparameter optimization accounts for overfitting through cross validation." - [https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74]

In [ ]:
#number of trees in random forest
n_estimators = [200, 400, 600, 1000, 1200, 1600, 1800, 2000]
#number of features to consider at every split
max_features = ['auto', 'sqrt']
#maximum number of levels in tree
max_depth = [20, 30, 40, 60, 80, 100]
#minimum number of samples required to split a node
min_samples_split = [2, 7]
#minimum number of samples required at each leaf node
min_samples_leaf = [2, 4]
#method of selecting samples for training each tree
bootstrap = [True, False] 

In [ ]:
#create the parameters grid for selecting in cross validation
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(param_grid)

In [ ]:
#construct random forest regression model using applying cross validation with grid of parameters
rfr2 = RandomForestRegressor()
rfr2_Grid = GridSearchCV(estimator = rfr2, param_grid = param_grid, cv = 3, verbose=2, n_jobs = 4)

In [ ]:
#fit to train data
rfr2_Grid.fit(X_train, y_train) #уменьшить число в параметрах

In [ ]:
rfr2_Grid.best_params_ #display most effective set of parameters

In [ ]:
rfr_tuned_model = RandomForestRegressor(bootstrap = True,max_depth= 4, max_features = 'auto',
                                        min_samples_leaf=2, min_samples_split=2,n_estimators=64)

In [ ]:
#train model using training dataset
rfr_tuned_model.fit(X_train,y_train)
print('Training score: {:0.3f}'.format(r2_score(y_train,rfr_tuned_model.predict(X_train))))
print('Training MSE: {:0.3f}'.format(mean_squared_error(y_train,rfr_tuned_model.predict(X_train))))
print('test set score: {:0.3f}'.format(r2_score(y_test,rfr_tuned_model.predict(X_test))))
print('test set MSE: {:0.3f}'.format(mean_squared_error(y_test,rfr_tuned_model.predict(X_test))))

in final result of using first machine learning algortihm - Random Forest Regressor using hyperparameter tunning & cross-validation techniques, We encountered overfitting - a frequent phenomenon when using the model. "The main problem is that they tend to overfit very easily. This causes high variance, which can be seen as high test errors on the test dataset, despite high accuracy on the training dataset. In other words, decision trees do not generalize well to novel data." - [https://www.keboola.com/blog/random-forest-regression#:~:text=Random%20forest%20can%20be%20used,predict%20categorical%20or%20discrete%20outputs]

In [ ]:
# Calculate feature importances
importances = rfr_tuned_model.feature_importances_

# Visualize Feature Importance
# Sort feature importances in descending order
indices = np.argsort(importances)[::-1]

# Rearrange feature names so they match the sorted feature importances
names = [X_train.columns[i] for i in indices]

plt.figure(figsize = (12, 5))
sns.set_style("whitegrid")
chart = sns.barplot(x = names, y=importances[indices])
plt.xticks(
    rotation=45, 
    horizontalalignment='right',
    fontweight='light'  
)
plt.title('Random forest. Feature importance')
plt.tight_layout()

by viewing the bar chart above, it can be seemed that Start and ending values of gas consumption are most important with Km feature. Least importance have diameter feature.

### Next model that we going to perform - XSGBoost or Extreme Gradient Boosting using hyperparameter tuning

In [ ]:
#create the parameters grid for selecting in cross validation
param_grid_xgb = {'n_estimators': [100, 200, 500,1000],
                  'max_depth': [4, 6, 8, 10],
                  'learning_rate': [0.01, 0.1, 0.5],
                  'min_child_weight':[3, 5, 7]
                 }

In [ ]:
xgb_model = XGBRegressor()
# set GridSearchCV parameters
xgb_model = GridSearchCV(estimator=xgb_model, param_grid=param_grid_xgb, verbose = 1, n_jobs = 4, cv = 3)

In [ ]:
#fit to train data
xgb_model.fit(X_train, y_train)

In [ ]:
xgb_model.best_params_ #display most effective set of parameters

In [ ]:
xgb_tuned_model = XGBRegressor(learning_rate = 0.01,max_depth= 8, min_child_weight = 7,n_estimators=500)

In [ ]:
#train model using training dataset
xgb_tuned_model.fit(X_train,y_train)
print('Training score: {:0.3f}'.format(r2_score(y_train,xgb_tuned_model.predict(X_train))))
print('Training MSE: {:0.3f}'.format(mean_squared_error(y_train,xgb_tuned_model.predict(X_train))))
print('test set score: {:0.3f}'.format(r2_score(y_test,xgb_tuned_model.predict(X_test))))
print('test set MSE: {:0.3f}'.format(mean_squared_error(y_test,xgb_tuned_model.predict(X_test))))

In [ ]:
# Calculate feature importances
importances = xgb_tuned_model.feature_importances_

# Visualize Feature Importance
# Sort feature importances in descending order
indices = np.argsort(importances)[::-1]

# Rearrange feature names so they match the sorted feature importances
names = [X_train.columns[i] for i in indices]

plt.figure(figsize = (12, 5))
sns.set_style("whitegrid")
chart = sns.barplot(x = names, y=importances[indices])
plt.xticks(rotation=45, horizontalalignment='right', fontweight='light')
plt.title('XGBoost. Feature importance')
plt.tight_layout()

Again, after using extreme gradient boosting algorithm, we got similar results as RF. Km, start and end rates of gas consumption are most important features while pressure end value is unnecessary.

### Compare models 

In [ ]:
models = []
# add our tuned models into list
models.append(('Random Forest', rfr_tuned_model))
models.append(('XGBoost', xgb_tuned_model))

results = []
names = []

# evaluate each model in turn
for name, model in models:
    kfold = KFold(n_splits=10, shuffle = True, random_state = 1)
    cv_results = cross_val_score(model, X_train.values, y_train.values, cv = 5, scoring = 'r2', n_jobs = -1)
    results.append(cv_results)
    names.append(name)
    # print mean accuracy and standard deviation
    print("%s: %f (%f)" % (name, cv_results.mean(), cv_results.std()))

In [ ]:
fig = plt.figure(figsize=(6,4))
plt.boxplot(results)
plt.title('Algorithm Comparison')
plt.xticks([1,2], names)
plt.show()

In [ ]:
pd.DataFrame({'predicted_value': xgb_tuned_model.predict(X_test), 'actual_value': y_test}, 
             columns=['predicted_value','actual_value'])

In [ ]:
pd.DataFrame({'predicted_value': rfr_tuned_model.predict(X_test), 'actual_value': y_test}, 
             columns=['predicted_value','actual_value'])

In [ ]:
xgb_tuned_model.predict(np.array([5.500000,989.7,1.9,9.5,17.1,0.4,1020,8.0,46.0]).reshape(1,-1))

In [ ]:
X_test

In [ ]:
user_input=input("Enter the values one by one")
user_input=user_input.split(",")


for i in range(len(user_input)):
    # convert each item to int type
    user_input[i] = float(user_input[i])

user_input=np.array(user_input)
user_input=user_input.reshape(1,-1)
xgb_y_pred=xgb_tuned_model.predict(user_input)
if(xgb_y_pred[0]==0):
  print("No gas leak detected at choosen pipeline")
else:
  print("Gas leak detected at",xgb_y_pred[0])

In [ ]:
#saving the best model using pickle
import pickle as pkl
pkl.dump(xgb_tuned_model,open("final_model.p","wb"))

## Web app building with streamlit library

In [ ]:
%%writefile gas_leak_prediction_app.py
import streamlit as st
import base64
import sklearn
import numpy as np
import pickle as pkl
#Load the saved model
model=pkl.load(open("final_model.p","rb"))

st.set_page_config(page_title="Gas Leak Prediction App",page_icon="⛽️",layout="centered",initial_sidebar_state="expanded")

def preprocess(Km,press_start,press_end,temp_start,temp_end,diameter,gas_consum_start,
               gas_consum_end,Davlenie_proektnoe):   
 
    
    # Pre-processing user input   
    if Km=='МГ "Макат-Северный Кавказ" ':
        Km=371.8
    elif Km=='Основная нитка. МГ ГШ':
        Km=309.0
    elif Km=='Основная нитка. МГ Жанажол - КС13':
        Km=157.4
    elif Km=='САЦ-4':
        Km=823.0
    elif Km=='МГ "САЦ-5" ':
        Km=823.0
    elif Km=='Основная нитка. МГ Союз':
        Km=382.0
    elif Km=='1 нитка. БГР-ТБА':
        Km=954.0
    elif Km=='2 нитка. БГР-ТБА':
        Km=954.0
    elif Km=='1 нитка. Бухара - Урал':
        Km=989.7
    elif Km=='2 нитка. Бухара - Урал':
        Km=989.7
    elif Km=='3 нитка. Бухара - Урал':
        Km=989.7
    
    
    if diameter == 'МГ "Макат-Северный Кавказ" - (1420)':
        diameter=1420
    elif diameter=='Основная нитка. МГ ГШ - (1220)':
        diameter=1220
    elif diameter=='Основная нитка. МГ Жанажол-КС13 - (813)':
        diameter=813
    elif diameter=='САЦ-4 - (1420)':
        diameter=1420
    elif diameter=='МГ "САЦ-5" - (1220)':
        diameter=1220
    elif diameter=='Основная нитка. МГ Союз - (1420)':
        diameter=1420
    elif diameter=='1 нитка. БГР-ТБА - (720)':
        diameter=720
    elif diameter=='2 нитка. БГР-ТБА - (1020)':
        diameter=1020
    elif diameter=='1 нитка. Бухара - Урал - (1020)':
        diameter=1020
    elif diameter=='2 нитка. Бухара - Урал - (1020)':
        diameter=1020
    elif diameter=='3 нитка. Бухара - Урал - (1020)':
        diameter=1020



    user_input=[Km,press_start,press_end,temp_start,temp_end,diameter,gas_consum_start,gas_consum_end,Davlenie_proektnoe]
    user_input=np.array(user_input)
    user_input=user_input.reshape(1,-1)
    prediction = model.predict(user_input)

    return prediction

    

       
    # front end elements of the web page 
html_temp = """ 
    <div style ="background-color:MediumAquamarine;padding:13px"> 
    <h1 style ="color:black;text-align:center;"> Gas Leak Prediction app</h1> 
    </div> 
    """
      
# display the front end aspect
st.markdown(html_temp, unsafe_allow_html = True) 
st.subheader('by Sabina & Zhanna')

# following lines create boxes in which user can enter data required to make prediction
Km=st.selectbox ("Name of the Pipeline",('МГ "Макат-Северный Кавказ','Основная нитка. МГ ГШ','Основная нитка. МГ Жанажол - КС1',
                                    'САЦ-4','МГ "САЦ-5" ','Основная нитка. МГ Союз','1 нитка. БГР-ТБА','2 нитка. БГР-ТБА',
                                     '1 нитка. Бухара - Урал','2 нитка. Бухара - Урал','3 нитка. Бухара - Урал'))
press_start=st.number_input('Pressure start value(kgf/cm2)',min_value=0, max_value=80)
press_end=st.number_input('Pressure end value(kgf/cm2)',min_value=0, max_value=80)
temp_start=st.number_input('Temperature start value',min_value=0, max_value=100)
temp_end=st.number_input('Temperature end value',min_value=0, max_value=100)
diameter=st.selectbox ("Name of the Pipeline",('МГ "Макат-Северный Кавказ" - (1420)','Основная нитка. МГ ГШ - (1220)',
                                               'Основная нитка. МГ Жанажол-КС13 - (813)','САЦ-4 - (1420)','МГ "САЦ-5" - (1220)',
                                               'Основная нитка. МГ Союз - (1420)','1 нитка. БГР-ТБА - (720)',
                                               '2 нитка. БГР-ТБА - (1020)','1 нитка. Бухара - Урал - (1020)',
                                               '2 нитка. Бухара - Урал - (1020)','3 нитка. Бухара - Урал - (1020)'))
gas_consum_start=st.number_input('Gas consumption start value',min_value=0, max_value=50)
gas_consum_end=st.number_input('Gas consumption end value',min_value=0, max_value=50)
Davlenie_proektnoe=st.number_input('Projected pressure value')



#user_input=preprocess(...)
pred=preprocess(Km,press_start,press_end,temp_start,temp_end,diameter,gas_consum_start,gas_consum_end,Davlenie_proektnoe)




if st.button("Predict"):    
    if pred[0] == 0:
        st.error('No gas leak detected at choosen pipeline')
    else:
        st.text_area('Gas leak detected at',pred[0])
    
   



st.sidebar.subheader("About App")

st.sidebar.info("This web app is helps you to find out whether at pipeline was happen a gas leak")
st.sidebar.info("Enter the required fields and click on the 'Predict' button to check whether gas leak is detected at pipeline")


feedback = st.sidebar.slider('How much would you rate this app?',min_value=0,max_value=5,step=1)

if feedback:
  st.header("Thank you for rating the app!")



In [ ]:
!nohup streamlit run gas-leak-prediction-app.py &
url = ngrok.connect(port='8501')
url